In [1]:
import os, sys, json 
from pathlib import Path

def find_src_folder():
    current = Path(os.getcwd()).resolve()
    for p in [current] + list(current.parents):
        src = p / "src"
        if src.exists():
            return src
    raise RuntimeError("src 폴더를 찾을 수 없습니다.")

src_path = find_src_folder()
sys.path.append(str(src_path))

from time import sleep
from langchain_openai import ChatOpenAI
from langchain.messages import SystemMessage, HumanMessage
from enums.LLM import LLM

maltus = [
    "친근한 반말",
    "버릇없는 반말",
    "공손한 존댓말",
    "AI에게 명령하는 투",
    "귀엽고 장난스러운 말투",
    "차분한 해설조 말투",
    "흥분한 플레이어 말투",
    "지친/피곤한 말투",
    "은근 반말 섞인 존댓말",
    "무뚝뚝하고 단문 위주의 말투",
    "주어가 명확하지 않은 말투",
    "기계 같은 말투",
    "싸가지 없는 말투",
]
llm = ChatOpenAI(model=LLM.GPT5_1, temperature=1.0)
from lab.fairy.dungeon_create_dataset_prompts import (
    get_system_prompt,
    HUMAN_CREATE_DATASET,
)
all_data = []
for maltu in maltus:
    res = llm.invoke(
        [
            SystemMessage(content=get_system_prompt(maltu)),
            HumanMessage(content=HUMAN_CREATE_DATASET),
        ]
    )
    raw = res.content
    try:
        data = json.loads(raw)
    except:
        # 혹시 모델이 쓰잘데기 넣으면 자동 정리
        try:
            json_str = raw[raw.index('['): raw.rindex(']')+1]
            data = json.loads(json_str)
        except Exception as e:
            print("❌ JSON 파싱 실패:", e)
            print("원본:", raw[:1000])
            
    all_data.extend(data)

output_path = "./fairy_dungeon_intent_dataset_total.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(all_data, f, ensure_ascii=False, indent=2)


/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
